# **Achieving 99.4% accuracy in MNIST using MLPs**

In [1]:
import torch
import torchvision
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as  F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

print("Torch version: ", torch. __version__)

####################################################################
# Set Device
####################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

Torch version:  2.10.0.dev20251124+cu130
Device:  cuda


In [3]:
####################################################################
# Prepare Data
####################################################################

train_set = torchvision.datasets.MNIST('.data/', train=True, download=True)
#train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_set = torchvision.datasets.MNIST('.data/', train=False, download=True)
#test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True)

print("Train images: ", train_set)
print("Image: ", train_set[0][0])
print("Label: ", train_set[0][1])
print("Label one hot: ", F.one_hot(torch.tensor(train_set[0][1]), num_classes=10))

Train images:  Dataset MNIST
    Number of datapoints: 60000
    Root location: .data/
    Split: Train
Image:  <PIL.Image.Image image mode=L size=28x28 at 0x222FB8CA410>
Label:  5
Label one hot:  tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])


In [4]:
####################################################################
# Dataset Class
####################################################################

class GaussianNoise(object):
    def __init__(self, std=0.05):
        self.std = std

    def __call__(self, tensor):
        return tensor + torch.randn_like(tensor) * self.std

class MNIST_dataset(Dataset):

    def __init__(self, data, partition = "train"):

        print("\nLoading MNIST ", partition, " Dataset...")
        self.data = data
        self.partition = partition
        self.transform_train = transforms.Compose([
            # Rotaciones suaves
            transforms.RandomRotation(10),
            # Desplazamientos y escalados
            transforms.RandomAffine(
                degrees=0,
                translate=(0.1, 0.1),
                scale=(0.9, 1.1)
            ),
            # Deformaciones elásticas
            transforms.ElasticTransform(alpha=15.0, sigma=5.0),
            transforms.ToTensor(),
            GaussianNoise(0.05),
            # Normalización
            transforms.Normalize((0.1307,), (0.3081,)),
        ])
        self.transform_test = transforms.Compose([
            transforms.ToTensor(),
            # Normalización
            transforms.Normalize((0.1307,), (0.3081,)),
        ])
        print("\tTotal Len.: ", len(self.data), "\n", 50*"-")

    # Métodos de herencia obligatorios __len__ y __item__
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Image
        image = self.data[idx][0]
        # PIL Image to torch tensor
        if self.partition == "train":
            image_tensor = self.transform_train(image)
        else:
            image_tensor = self.transform_test(image)
        # care! net expect a 784 size vector and our dataset
        # provide 1x28x28 (channels, height, width) -> Reshape!
        # Con el view -1 pasamos de 28x28 (28, 28) a 764 (, 764)
        image_tensor = image_tensor.view(-1)

        # Label
        label = torch.tensor(self.data[idx][1])
        label = F.one_hot(label, num_classes=10).float()

        return {"img": image_tensor, "label": label}

train_dataset = MNIST_dataset(train_set, partition="train")
test_dataset = MNIST_dataset(test_set, partition="test")


Loading MNIST  train  Dataset...
	Total Len.:  60000 
 --------------------------------------------------

Loading MNIST  test  Dataset...
	Total Len.:  10000 
 --------------------------------------------------


In [5]:
####################################################################
# DataLoader Class
####################################################################

batch_size = 100
# num_workers = multiprocessing.cpu_count()-1
num_workers = 0
print("Num workers", num_workers)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

Num workers 0


In [6]:
####################################################################
# Neural Network Class
####################################################################

# Creating our Neural Network - Fully Connected
# Hereda de nn.Module
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        # Inicio
        self.linear1 = nn.Linear(784, 1024)
        self.batch_norm1 = nn.BatchNorm1d(1024)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)

        # BLoque residual estándar
        self.residual_bloq1 = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024)
        )

        # BLoque residual estándar
        self.residual_bloq2 = nn.Sequential(
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(1024, 1024),
            nn.BatchNorm1d(1024)
        )
        

        self.classifier = nn.Linear(1024, num_classes)

    # Determina el orden de las capas
    def forward(self, x):
        # Capa inicial
        out = self.linear1(x)
        out = self.batch_norm1(out)
        out = self.relu1(out)
        out = self.dropout1(out)

        # Bloque residual 1
        residual = out
        out = self.residual_bloq1(out)
        out = out + residual
        out = torch.relu(out) # Activación después de la residual

        # Bloque residual 2
        residual = out
        out = self.residual_bloq2(out)
        out = out + residual
        out = torch.relu(out) # Activación después de la residual

        # Clasificador
        out = self.classifier(out)
        return out



# Instantiating the network and printing its architecture
num_classes = 10
net = Net(num_classes)
print(net)

# Cuenta parámetros entrenables
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Params: ", count_parameters(net))

Net(
  (linear1): Linear(in_features=784, out_features=1024, bias=True)
  (batch_norm1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.2, inplace=False)
  (residual_bloq1): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=1024, out_features=1024, bias=True)
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (residual_bloq2): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=1024, out_features=1024, bias=True)
    (5): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=T

In [6]:
####################################################################
# Training settings
####################################################################

# Training hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9)
# scheduler (OneCycleLR)
# steps = longitud del dataset / tamaño de batch
epochs = 100

In [7]:
####################################################################
# Training
####################################################################

# Load model in GPU
net.to(device)

print("\n---- Start Training ----")
best_accuracy = -1
best_epoch = 0
for epoch in range(epochs):


    # TRAIN NETWORK
    train_loss, train_correct = 0, 0
    # Modo de entrenamiento del modelo
    net.train()
    with tqdm(iter(train_dataloader), desc="Epoch " + str(epoch), unit="batch") as tepoch:
        for batch in tepoch:

            # Returned values of Dataset Class
            images = batch["img"].to(device)
            labels = batch["label"].to(device)

            # zero the parameter gradients
            # Por cada step, reiniciar los gradientes para que no haga max
            optimizer.zero_grad()

            # Forward
            outputs = net(images)
            loss = criterion(outputs, labels)

            # Calculate gradients
            loss.backward()

            # Update gradients
            optimizer.step()

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)
            train_correct += pred.eq(labels).sum().item()

            # print statistics
            train_loss += loss.item()

    train_loss /= len(train_dataloader.dataset)

    # TEST NETWORK
    test_loss, test_correct = 0, 0
    net.eval()
    with torch.no_grad():
      with tqdm(iter(test_dataloader), desc="Test " + str(epoch), unit="batch") as tepoch:
          for batch in tepoch:

            images = batch["img"].to(device)
            labels = batch["label"].to(device)

            # Forward
            outputs = net(images)
            test_loss += criterion(outputs, labels)

            # one hot -> labels
            labels = torch.argmax(labels, dim=1)
            pred = torch.argmax(outputs, dim=1)

            test_correct += pred.eq(labels).sum().item()

    test_loss /= len(test_dataloader.dataset)
    test_accuracy = 100. * test_correct / len(test_dataloader.dataset)

    print("[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(
        epoch + 1, train_loss, test_loss, 100. * train_correct / len(train_dataloader.dataset), test_accuracy
    ))

    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        best_epoch = epoch

        # Save best weights
        torch.save(net.state_dict(), "best_model.pt")

print("\nBEST TEST ACCURACY: ", best_accuracy, " in epoch ", best_epoch)


---- Start Training ----


Test 0: 100%|██████████| 100/100 [00:02<00:00, 40.33batch/s]


[Epoch 1] Train Loss: 0.005713 - Test Loss: 0.001174 - Train Accuracy: 82.63% - Test Accuracy: 96.25%


Test 1: 100%|██████████| 100/100 [00:02<00:00, 41.40batch/s]


[Epoch 2] Train Loss: 0.003107 - Test Loss: 0.000827 - Train Accuracy: 90.64% - Test Accuracy: 97.39%


...

[Epoch 98] Train Loss: 0.000511 - Test Loss: 0.000179 - Train Accuracy: 98.37% - Test Accuracy: 99.46%


Test 98: 100%|██████████| 100/100 [00:02<00:00, 38.96batch/s]


[Epoch 99] Train Loss: 0.000520 - Test Loss: 0.000197 - Train Accuracy: 98.36% - Test Accuracy: 99.40%


Test 99: 100%|██████████| 100/100 [00:02<00:00, 38.97batch/s]

[Epoch 100] Train Loss: 0.000499 - Test Loss: 0.000230 - Train Accuracy: 98.38% - Test Accuracy: 99.31%

BEST TEST ACCURACY:  99.47  in epoch  65


In [8]:
import torch 
print("Torch:", torch.__version__)

Torch: 2.10.0.dev20251124+cu130
